# Barber Shop Booking Agent - Capstone Project

## Overview
This notebook demonstrates a Multi-Agent System for a Barber Shop, designed for the 'Enterprise Agents' track. It handles:
- Appointment Booking (Mock Calendar)
- Style Consultation (Mock Vision)
- Client Follow-up

## Architecture
- **Orchestrator**: Main loop routing based on `AgentState`.
- **Agents**: Receptionist, Booking, Consultation, Follow-up.
- **Tools**: MockCalendarTool.


In [ ]:
import datetime
import json
from typing import TypedDict, List, Optional, Dict, Any

# --- State Definition ---
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    next_agent: Optional[str]
    client_data: Dict[str, Any]
    current_task: Optional[str]
    error: Optional[str]

# --- Tools ---
class MockCalendarTool:
    def __init__(self):
        self.events = []
        today = datetime.date.today()
        self.events.append({
            "id": "evt_1",
            "summary": "Haircut - John Doe",
            "start": f"{today}T10:00:00",
            "end": f"{today}T10:45:00",
            "status": "confirmed"
        })
        self.events.append({
            "id": "evt_2",
            "summary": "Beard Trim - Mike Smith",
            "start": f"{today}T14:00:00",
            "end": f"{today}T14:30:00",
            "status": "confirmed"
        })

    def list_events(self, date_str: str) -> str:
        found_events = [e for e in self.events if e["start"].startswith(date_str)]
        if not found_events:
            return f"No events found for {date_str}."
        result = f"Events for {date_str}:\n"
        for evt in found_events:
            result += f"- [{evt['start'].split('T')[1]} - {evt['end'].split('T')[1]}] {evt['summary']} (ID: {evt['id']})\n"
        return result

    def check_availability(self, date_str: str, time_str: str) -> bool:
        requested_start = f"{date_str}T{time_str}"
        for evt in self.events:
            if evt["start"] == requested_start:
                return False
        return True

    def create_event(self, summary: str, date_str: str, time_str: str, duration_minutes: int = 45) -> str:
        if not self.check_availability(date_str, time_str):
            return f"Error: Slot {date_str} {time_str} is already booked."
        new_id = f"evt_{len(self.events) + 1}"
        start_dt = datetime.datetime.strptime(f"{date_str}T{time_str}", "%Y-%m-%dT%H:%M:%S")
        end_dt = start_dt + datetime.timedelta(minutes=duration_minutes)
        new_event = {
            "id": new_id,
            "summary": summary,
            "start": f"{date_str}T{time_str}",
            "end": end_dt.strftime("%Y-%m-%dT%H:%M:%S"),
            "status": "confirmed"
        }
        self.events.append(new_event)
        return f"Success: Booked '{summary}' for {date_str} at {time_str}. (ID: {new_id})"

# --- Agents ---
class ReceptionistAgent:
    def run(self, state: AgentState) -> AgentState:
        print("--- Receptionist Agent ---")
        last_message = state["messages"][-1]["content"].lower()
        if "book" in last_message or "schedule" in last_message:
            state["next_agent"] = "booking_agent"
        elif "style" in last_message or "photo" in last_message:
            state["next_agent"] = "consultation_agent"
        elif "feedback" in last_message:
            state["next_agent"] = "followup_agent"
        else:
            state["next_agent"] = None
            state["messages"].append({"role": "assistant", "content": "Welcome! I can help you book, consult on style, or leave feedback."})
        return state

class BookingAgent:
    def __init__(self, calendar_tool):
        self.calendar = calendar_tool
    def run(self, state: AgentState) -> AgentState:
        print("--- Booking Agent ---")
        last_message = state["messages"][-1]["content"]
        today = datetime.date.today().isoformat()
        if " at " in last_message or ":" in last_message:
            try:
                words = last_message.split()
                time_str = "10:00:00"
                for w in words:
                    if ":" in w:
                        time_str = w
                        if len(time_str) == 5: time_str += ":00"
                        break
                result = self.calendar.create_event("Client Appointment", today, time_str)
                state["messages"].append({"role": "assistant", "content": result})
                state["next_agent"] = "receptionist_agent"
            except Exception as e:
                state["messages"].append({"role": "assistant", "content": f"Error: {str(e)}"})
        else:
            events = self.calendar.list_events(today)
            state["messages"].append({"role": "assistant", "content": f"{events}\nWhat time? (e.g., 'at 10:00')"})
            state["next_agent"] = "booking_agent"
        return state

class ConsultationAgent:
    def run(self, state: AgentState) -> AgentState:
        print("--- Consultation Agent ---")
        last_message = state["messages"][-1]["content"].lower()
        if "photo" in last_message:
            state["messages"].append({"role": "assistant", "content": "Gemini Vision Analysis: Textured Crop. Book this?"})
            state["client_data"]["preferred_style"] = "Textured Crop"
        elif "yes" in last_message:
             state["next_agent"] = "booking_agent"
             state["messages"].append({"role": "assistant", "content": "Great! Let's book it."})
        else:
            state["messages"].append({"role": "assistant", "content": "Upload a photo or describe the look."})
            state["next_agent"] = "consultation_agent"
        return state

class FollowupAgent:
    def run(self, state: AgentState) -> AgentState:
        print("--- Follow-up Agent ---")
        state["messages"].append({"role": "assistant", "content": "Thanks for the feedback! Share a selfie?"})
        state["next_agent"] = "receptionist_agent"
        return state

# --- Main ---
def run_agent_interactive():
    calendar_tool = MockCalendarTool()
    receptionist = ReceptionistAgent()
    booking = BookingAgent(calendar_tool)
    consultation = ConsultationAgent()
    followup = FollowupAgent()
    
    state = {"messages": [], "next_agent": "receptionist_agent", "client_data": {}, "current_task": None, "error": None}
    
    print("Start chatting (type 'quit' to stop)...")
    while True:
        if not state["messages"] or state["messages"][-1]["role"] == "assistant":
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit"]:
                break
            state["messages"].append({"role": "user", "content": user_input})
        
        current = state["next_agent"]
        if current == "receptionist_agent": state = receptionist.run(state)
        elif current == "booking_agent": state = booking.run(state)
        elif current == "consultation_agent": state = consultation.run(state)
        elif current == "followup_agent": state = followup.run(state)
        else: state["next_agent"] = "receptionist_agent"
        
        if state["messages"] and state["messages"][-1]["role"] == "assistant":
            print(f"Agent: {state['messages'][-1]['content']}")

# Uncomment to run in Colab:
# run_agent_interactive()
